### 트랜스포머 구성하기

In [130]:
import pandas as pd
import numpy as np
import string

In [131]:
lines= pd.read_table('english to french.txt', names=['eng', 'fr'])

In [132]:
lines = lines[0:50000]
print(lines.head(3))
print(lines.tail(3))

    eng        fr
0   Go.      Va !
1  Run!   Cours !
2  Run!  Courez !
                            eng                              fr
49997  They go to work on foot.     Ils vont au travail à pied.
49998  They got into the train.    Ils montèrent dans le train.
49999  They got into the train.  Elles montèrent dans le train.


In [133]:
lines['eng']=lines['eng'].apply(lambda x: x.lower())
lines['fr']=lines['fr'].apply(lambda x: x.lower())

In [134]:
exclude = set(string.punctuation)
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.fr=lines.fr.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [135]:
print(lines.head(3))
print(lines.tail(3))

   eng       fr
0   go      va 
1  run   cours 
2  run  courez 
                           eng                             fr
49997  they go to work on foot     ils vont au travail à pied
49998  they got into the train    ils montèrent dans le train
49999  they got into the train  elles montèrent dans le train


In [136]:
lines.fr = lines.fr.apply(lambda x : 'start '+ x + ' end')

In [137]:
print(lines.head(3))
print(lines.tail(3))

   eng                 fr
0   go      start va  end
1  run   start cours  end
2  run  start courez  end
                           eng                                       fr
49997  they go to work on foot     start ils vont au travail à pied end
49998  they got into the train    start ils montèrent dans le train end
49999  they got into the train  start elles montèrent dans le train end


In [138]:
lines.shape

(50000, 2)

In [139]:
# fit a tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [140]:
import json
eng_tokenizer = create_tokenizer(lines['eng'])
eng_dict=json.loads(json.dumps(eng_tokenizer.word_counts))
df =pd.DataFrame([eng_dict.keys(), eng_dict.values()]).T
df.columns = ['word','count']

df = df.sort_values(by='count',ascending = False)
df['cum_count']=df['count'].cumsum()
df['cum_perc'] = df['cum_count']/df['cum_count'].max()
final_eng_words = df[df['cum_perc']<0.8]['word'].values
#final_eng_words=df['word'].values

In [141]:
fr_tokenizer = create_tokenizer(lines['fr'])
fr_dict = json.loads(json.dumps(fr_tokenizer.word_counts))
df =pd.DataFrame([fr_dict.keys(), fr_dict.values()]).T
df.columns = ['word','count']
df = df.sort_values(by='count',ascending = False)
df['cum_count']=df['count'].cumsum()
df['cum_perc'] = df['cum_count']/df['cum_count'].max()
final_fr_words = df[df['cum_perc']<0.8]['word'].values
#final_fr_words=df['word'].values

In [142]:
print(len(final_eng_words),len(final_fr_words))

384 357


In [143]:
def filter_eng_words(x):
  t = []
  x = x.split()
  for i in range(len(x)):
    if x[i] in final_eng_words:
      t.append(x[i])
    else:
      t.append('unk')
  x3 = ''
  for i in range(len(t)):
    x3 = x3+t[i]+' '
  return x3

In [144]:
filter_eng_words('he is extremely good')

'he is unk good '

In [145]:
def filter_fr_words(x):
  t = []
  x = x.split()
  for i in range(len(x)):
    if x[i] in final_fr_words:
      t.append(x[i])
    else:
      t.append('unk')
  x3 = ''
  for i in range(len(t)):
    x3 = x3+t[i]+' '
  return x3

In [146]:
lines['eng']=lines['eng'].apply(filter_eng_words)
lines['fr']=lines['fr'].apply(filter_fr_words)

In [147]:
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
    
all_french_words=set()
for fr in lines.fr:
    for word in fr.split():
        if word not in all_french_words:
            all_french_words.add(word)

In [148]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_french_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_french_words)
# del all_eng_words, all_french_words

In [149]:
set(all_french_words) - set(final_fr_words) 

{'unk'}

In [150]:
len(all_eng_words)

385

In [151]:
len(target_words)

358

In [152]:
input_token_index = dict(
    [(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i+1) for i, word in enumerate(target_words)])

In [153]:
num_decoder_tokens

358

In [154]:
print(target_token_index['start']) 
print(target_token_index['end']) 
print(list(input_token_index.keys())[384])
print(list(target_token_index.keys())[283])
print(list(target_token_index.keys())[88])

284
89
youve
start
end


In [155]:
length_list=[]
for l in lines.fr:
    length_list.append(len(l.split(' ')))
fr_max_length = np.max(length_list)

In [156]:
length_list=[]
for l in lines.eng:
    length_list.append(len(l.split(' ')))
eng_max_length = np.max(length_list)

In [157]:
print(eng_max_length)
print(fr_max_length)

8
17


In [158]:
encoder_input_data = np.zeros(
    (len(lines['eng']), eng_max_length),
    dtype='float32')
decoder= np.zeros(
    (len(lines['fr']), fr_max_length+1),
    dtype='float32')


In [159]:
decoder.shape

(50000, 18)

In [160]:
for i, (input_text, target_text) in enumerate(zip(lines['eng'], lines['fr'])):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        decoder[i, t] = target_token_index[word]
decoder_input_data=decoder[:,:-1]
decoder_target_data=decoder[:,1:]

In [161]:
print(encoder_input_data.shape,decoder_input_data.shape,decoder_target_data.shape)

(50000, 8) (50000, 17) (50000, 17)


In [162]:
print(encoder_input_data[1000])
print(decoder_input_data[1000])
print(decoder_target_data[1000])

[63.  8.  0.  0.  0.  0.  0.  0.]
[284. 320. 274.  89.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.]
[320. 274.  89.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.]


In [163]:
import tensorflow as tf
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [164]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)#(1,position,d_model)


In [165]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  #x:tokenized words with padding for each sentence 1D텐서 [...,0,0,0]
  # add extra dimensions to add the padding
  # to the attention logits.
  return mask[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [166]:
def create_look_ahead_mask(x):
    seq_len=tf.shape(x)[1]
    look_ahead_mask=1-tf.linalg.band_part(tf.ones((seq_len,seq_len)),-1,0)
    padding_mask=create_padding_mask(x)
    return tf.maximum(look_ahead_mask,padding_mask)

##x:tokenized words with padding for each sentence 1D텐서 [...,0,0,0](ts,), return (ts,ts)

In [167]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth) (number of heads,ts,64)
    k: key shape == (..., seq_len_k, depth)   (number of heads,ts,64)
    v: value shape == (..., seq_len_v, depth_v) (number of heads,ts,64)
    mask: Float tensor with shape broadcastable  (1,1,ts) or (ts,ts)
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k) (number of heads,ts,ts)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v) (number of heads,ts,64)

  return output, attention_weights

In [168]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads  #8
    self.d_model = d_model #512

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model) #(ts,512)
    self.wk = tf.keras.layers.Dense(d_model) #(ts,512)
    self.wv = tf.keras.layers.Dense(d_model) #(ts,512)

    self.dense = tf.keras.layers.Dense(d_model) # scale_dot_attention후 concatenate한 attention (ts,512)를 Dense에 통과하기 위함

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    (batch,ts,512)를 (batch,ts,heads(8), depth(64))로 reshape한후 (batch,heads,ts,depth)로 반환하는 함수
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, ts, 512) 여기에서 512=d_model=embedding_size
    k = self.wk(k)  # (batch_size, ts, 512)
    v = self.wv(v)  # (batch_size, ts, 512)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth) depth=64, num_heads=8,seq_len_q=ts 
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth) depth=64, num_heads=8,seq_len_k=ts 
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth) depth=64, num_heads=8,seq_len_v=ts 

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

In [169]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff) seq_len=ts
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model) d_model=512
  ])

In [170]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model) input_seq_len=input_ts, d_model=512
    attn_output = self.dropout1(attn_output)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

In [171]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model) x: decoder input

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model) 
    attn1 = self.dropout1(attn1)                   #targer_seq_len=decoder_input_ts
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
    attn2 = self.dropout2(attn2)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model) #target_seq_len=decoder_input_ts

    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

    return out3, attn_weights_block1, attn_weights_block2

In [172]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding,
                                            self.d_model)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, mask):

    seq_len = tf.shape(x)[1]  #x는 tokenized sentence with padding

    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, mask)

    return x  # (batch_size, input_seq_len, d_model)

In [173]:
a={}
for i in range(2):
    block1=i**2
    block2=i**3
    a["a_{}".format(i+1)]=block1
    a["b_{}".format(i+1)]=block2
print(a)   

{'a_1': 0, 'b_1': 0, 'a_2': 1, 'b_2': 1}


In [174]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output,look_ahead_mask, padding_mask)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

In [175]:
def transformer(num_layers, d_model, num_heads, dff, input_vocab_size,
               target_vocab_size, ts_input, ts_target, rate=0.1):
    
    enc_input = tf.keras.Input(shape=(None,))  
    dec_input = tf.keras.Input(shape=(None,))
    
    enc_padding_mask=tf.keras.layers.Lambda(create_padding_mask,output_shape=(1,1,None))(enc_input)
    look_ahead_mask=tf.keras.layers.Lambda(create_look_ahead_mask,output_shape=(1,None,None))(dec_input)
    dec_padding_mask=tf.keras.layers.Lambda(create_padding_mask,output_shape=(1,1,None))(enc_input)
    
    enc_output=Encoder(num_layers, d_model, num_heads, dff,
                             input_vocab_size, ts_input, rate)(enc_input,enc_padding_mask)
    dec_output,_ =Decoder(num_layers, d_model, num_heads, dff,
                           target_vocab_size, ts_target, rate)(dec_input,enc_output,look_ahead_mask,dec_padding_mask)
    outputs=tf.keras.layers.Dense(target_vocab_size)(dec_output)
    
    return tf.keras.Model(inputs=[enc_input, dec_input], outputs=outputs)
    


In [176]:
import tensorflow as tf
a=[False,False,True]
b=[False,True,True]
c=tf.math.logical_and(a,b)
print(c)

tf.Tensor([False False  True], shape=(3,), dtype=bool)


In [177]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [178]:
def loss_function(y_true, y_pred):
  mask = tf.math.logical_not(tf.math.equal(y_true, 0))
  loss_ = loss_object(y_true, y_pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask) #padding을 제외한 loss


def accuracy_function(y_true, y_pred):
  y_hat=tf.cast(tf.argmax(y_pred,axis=2), dtype=tf.float32) #y_true가 float32이기 때문
  accuracies = tf.equal(y_true, y_hat)
  #accuracies = tf.equal(y_true, tf.argmax(y_pred, axis=2)) #True, False로 출력

  mask = tf.math.logical_not(tf.math.equal(y_true, 0))
  accuracies = tf.math.logical_and(mask, accuracies) #둘다 true일 때만 True

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

def accuracy(y_true,y_pred):
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


In [179]:
import tensorflow as tf
data_set=tf.data.Dataset.from_tensor_slices(((encoder_input_data,decoder_input_data),decoder_target_data))
data_final=data_set.shuffle(1000).batch(64).prefetch(64)#memory를 차지하지 않음

In [180]:
model=transformer(num_layers=1,d_model=128,num_heads=4,dff=256, 
                  input_vocab_size=386, target_vocab_size=359,ts_input=8,ts_target=17,rate=0.2)

optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy_function])
model.fit(data_final,epochs=1)

782/782 [==============================] - 74s 88ms/step - loss: 2.0121 - accuracy_function: 0.5657


### 챗봇 테스트

In [183]:
!pip install korpora
from Korpora import Korpora
Korpora.corpus_list()

{'aihub_conversation_translation': 'AI Hub 에서 제공하는 번역용 병렬 말뭉치 (대화)',
 'aihub_decree_translation': 'AI Hub 에서 제공하는 번역용 병렬 말뭉치 (조례)',
 'aihub_government_website_translation': 'AI Hub 에서 제공하는 번역용 병렬 말뭉치 (지자체웹사이트)',
 'aihub_korean_culture_translation': 'AI Hub 에서 제공하는 번역용 병렬 말뭉치 (한국문화)',
 'aihub_news_translation': 'AI Hub 에서 제공하는 번역용 병렬 말뭉치 (뉴스)',
 'aihub_spoken_translation': 'AI Hub 에서 제공하는 번역용 병렬 말뭉치 (구어)',
 'aihub_translation': 'AI Hub 에서 제공하는 번역용 병렬 말뭉치 (구어 + 대화 + 뉴스 + 한국문화 + 조례 + 지자체웹사이트)',
 'kcbert': 'beomi@github 님이 만드신 KcBERT 학습데이터',
 'korean_chatbot_data': 'songys@github 님이 만드신 챗봇 문답 데이터',
 'korean_hate_speech': '{inmoonlight,warnikchow,beomi}@github 님이 만드신 혐오댓글데이터',
 'korean_parallel_koen_news': 'jungyeul@github 님이 만드신 병렬 말뭉치',
 'korean_petitions': 'lovit@github 님이 만드신 2017.08 ~ 2019.03 청와대 청원데이터',
 'kornli': 'KakaoBrain 에서 제공하는 Natural Language Inference (NLI) 데이터',
 'korsts': 'KakaoBrain 에서 제공하는 Semantic Textual Similarity (STS) 데이터',
 'kowikitext': 'lovit@github 님이 만드신 wikitex

In [186]:
import pandas as pd
chat_data=pd.read_csv('ChatbotData.csv')

In [187]:
print(chat_data.head())

                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0


In [188]:
print(chat_data.shape)

(11823, 3)


In [189]:
import string
exclude=set(string.punctuation)
chat_data.Q=chat_data.Q.apply(lambda x:''.join (ch for ch in x if ch not in exclude))
chat_data.A=chat_data.A.apply(lambda x:''.join (ch for ch in x if ch not in exclude))
print(chat_data.head())

                 Q           A  label
0            12시 땡   하루가 또 가네요      0
1      1지망 학교 떨어졌어    위로해 드립니다      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠      0
4          PPL 심하네   눈살이 찌푸려지죠      0


In [190]:
#!pip install tensorflow_datasets
import tensorflow_datasets as tfds
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    chat_data.Q+chat_data.A, target_vocab_size=2**13)
print(tokenizer.vocab_size)

8510


In [191]:
ex_1=tokenizer.encode(chat_data.loc[20,'Q'])
print(ex_1)
ex_2=tokenizer.decode(ex_1)
print(ex_2)

[5871, 590, 3523, 127, 695, 3756, 833]
가스비 비싼데 감기 걸리겠어


In [192]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

VOCAB_SIZE = tokenizer.vocab_size + 2 #2개의 token이 추가됨
print(VOCAB_SIZE)

8512


In [193]:
tokenized_inputs, tokenized_outputs = [], []

for (sentence1, sentence2) in zip(chat_data.Q, chat_data.A):
    # decoder(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
    sentence1 = tokenizer.encode(sentence1)
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    tokenized_inputs.append(sentence1)
    tokenized_outputs.append(sentence2) 

print(tokenized_inputs[:5])
print(tokenized_outputs[:5])

[[8251, 1153, 3075], [8303, 47, 906, 8286, 976, 1715], [8305, 1433, 4683, 8286, 3666, 87], [8305, 1433, 4683, 8286, 1284, 3666, 87], [8334, 8334, 8330, 8286, 4213]]
[[8510, 3857, 64, 8230, 8511], [8510, 1824, 5586, 8511], [8510, 3409, 762, 120, 8511], [8510, 3409, 762, 120, 8511], [8510, 964, 2300, 1490, 2180, 4436, 40, 8511]]


In [194]:
import numpy as np
lenth_list=[]
for l in tokenized_inputs:
    lenth_list.append(len(l))
q_lenth_max=np.max(lenth_list)
print(q_lenth_max)

lenth_list=[]
for l in tokenized_outputs:
    lenth_list.append(len(l))
a_lenth_max=np.max(lenth_list)
print(a_lenth_max)


20
29


In [195]:
import tensorflow as tf

max_length_q=20
questions = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=max_length_q, padding='post')

max_length_a=29
answers = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=max_length_a, padding='post')

In [196]:
print(questions[:2])
print(answers[:2])

[[8251 1153 3075    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [8303   47  906 8286  976 1715    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]]
[[8510 3857   64 8230 8511    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [8510 1824 5586 8511    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]]


In [197]:
print(tokenizer.decode(questions[0]))
print(questions.shape)
print(answers.shape)

12시 땡
(11823, 20)
(11823, 29)


In [198]:
import tensorflow as tf
data_set=tf.data.Dataset.from_tensor_slices(((questions,answers[:,:-1]),answers[:,1:]))
data_final=data_set.shuffle(500).batch(32).prefetch(32)
ds_test=next(iter(data_final))
print(np.array(ds_test[0][0].shape))
print(np.array(ds_test[1]).shape)
print(np.array(ds_test[0][1].shape))

[32 20]
(32, 28)
[32 28]


In [199]:
import tensorflow as tf
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [200]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)#(1,position,d_model)

In [201]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  #x:tokenized words with padding for each sentence 1D텐서 [...,0,0,0]
  # add extra dimensions to add the padding
  # to the attention logits.
  return mask[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [202]:
def create_look_ahead_mask(x):
    seq_len=tf.shape(x)[1]
    look_ahead_mask=1-tf.linalg.band_part(tf.ones((seq_len,seq_len)),-1,0)
    padding_mask=create_padding_mask(x)
    return tf.maximum(look_ahead_mask,padding_mask)

In [203]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth) (number of heads,ts,64)
    k: key shape == (..., seq_len_k, depth)   (number of heads,ts,64)
    v: value shape == (..., seq_len_v, depth_v) (number of heads,ts,64)
    mask: Float tensor with shape broadcastable  (1,1,ts) or (ts,ts)
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k) (number of heads,ts,ts)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v) (number of heads,ts,64)

  return output, attention_weights

In [204]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads  #8
    self.d_model = d_model #512

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model) #(ts,512)
    self.wk = tf.keras.layers.Dense(d_model) #(ts,512)
    self.wv = tf.keras.layers.Dense(d_model) #(ts,512)

    self.dense = tf.keras.layers.Dense(d_model) # scale_dot_attention후 concatenate한 attention (ts,512)를 Dense에 통과하기 위함

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    (batch,ts,512)를 (batch,ts,heads(8), depth(64))로 reshape한후 (batch,heads,ts,depth)로 반환하는 함수
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, ts, 512) 여기에서 512=d_model=embedding_size
    k = self.wk(k)  # (batch_size, ts, 512)
    v = self.wv(v)  # (batch_size, ts, 512)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth) depth=64, num_heads=8,seq_len_q=ts 
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth) depth=64, num_heads=8,seq_len_k=ts 
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth) depth=64, num_heads=8,seq_len_v=ts 

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

In [205]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff) seq_len=ts
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model) d_model=512
  ])

In [206]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model) input_seq_len=input_ts, d_model=512
    attn_output = self.dropout1(attn_output)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

In [207]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model) x: decoder input

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model) 
    attn1 = self.dropout1(attn1)                   #targer_seq_len=decoder_input_ts
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask) 
    attn2 = self.dropout2(attn2)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model) #target_seq_len=decoder_input_ts

    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

    return out3, attn_weights_block1, attn_weights_block2

In [208]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding,
                                            self.d_model)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, mask):

    seq_len = tf.shape(x)[1]  #x는 tokenized sentence with padding

    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, mask)

    return x  # (batch_size, input_seq_len, d_model)

In [209]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output,look_ahead_mask, padding_mask)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

In [210]:
def transformer(num_layers, d_model, num_heads, dff, input_vocab_size,
               target_vocab_size, ts_input, ts_target, rate=0.1):
    
    enc_input = tf.keras.Input(shape=(None,))  
    dec_input = tf.keras.Input(shape=(None,))
    
    enc_padding_mask=tf.keras.layers.Lambda(create_padding_mask,output_shape=(1,1,None))(enc_input)
    look_ahead_mask=tf.keras.layers.Lambda(create_look_ahead_mask,output_shape=(1,None,None))(dec_input)
    dec_padding_mask=tf.keras.layers.Lambda(create_padding_mask,output_shape=(1,1,None))(enc_input)
    
    enc_output=Encoder(num_layers, d_model, num_heads, dff,
                             input_vocab_size, ts_input, rate)(enc_input,enc_padding_mask)
    dec_output,_ =Decoder(num_layers, d_model, num_heads, dff,
                           target_vocab_size, ts_target, rate)(dec_input,enc_output,look_ahead_mask,dec_padding_mask)
    outputs=tf.keras.layers.Dense(target_vocab_size)(dec_output)
    
    return tf.keras.Model(inputs=[enc_input, dec_input], outputs=outputs)
    

In [211]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [212]:
def loss_function(y_true, y_pred):
  mask = tf.math.logical_not(tf.math.equal(y_true, 0))
  loss_ = loss_object(y_true, y_pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask) #padding을 제외한 loss


def accuracy_function(y_true, y_pred):
  y_hat=tf.cast(tf.argmax(y_pred,axis=2), dtype=tf.float32) #y_true가 float32이기 때문
  accuracies = tf.equal(y_true, y_hat)
  #accuracies = tf.equal(y_true, tf.argmax(y_pred, axis=2)) #True, False로 출력

  mask = tf.math.logical_not(tf.math.equal(y_true, 0))
  accuracies = tf.math.logical_and(mask, accuracies) #둘다 true일 때만 True

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

def accuracy(y_true,y_pred):
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


In [ ]:
model=transformer(num_layers=1,d_model=128,num_heads=4,dff=256, 
                  input_vocab_size=VOCAB_SIZE-2, target_vocab_size=VOCAB_SIZE,ts_input=20,ts_target=28,rate=0.1)
                  #num_layers=2,3,4는 작동안함. 아마도 작으면 encoder_input에 영향을 많이 받도록함
                  #num_layers가 크면 encoder보다 decoder_input에 더 큰영향을 받아 첫 word=START_TOKEN으로 같으므로 
                  #항상 같은 답변으로 나올 가능성이 많음. 실제로 그러함. embedding size는 작은 것이 우수함 
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy_function])
model.fit(data_final,epochs=3)

Epoch 1/3
370/370 [==============================] - 123s 300ms/step - loss: 6.5185 - accuracy_function: 0.1978
Epoch 2/3
370/370 [==============================] - 111s 300ms/step - loss: 5.0718 - accuracy_function: 0.2697
Epoch 3/3
 85/370 [=====>........................] - ETA: 1:25 - loss: 3.9909 - accuracy_function: 0.3723

In [ ]:
def evaluate(sentence):
  sentence_1 = preprocess_sentence(sentence)
  token_sentence=[]
  t_sentence=tokenizer.encode(sentence_1)
  token_sentence.append(t_sentence)
  encoder_input=tf.keras.preprocessing.sequence.pad_sequences(token_sentence, maxlen=20, padding='post')
  print(encoder_input)
  decoder_input_pred=np.zeros((1,max_length_a-1),dtype='int32')
  word=START_TOKEN[0]
  for i in range(max_length_a-1):
     decoder_input_pred[:,i]=word
     pred=model.predict((encoder_input,decoder_input_pred))
     t=np.argmax(pred[0][i])
     word=t
     if word == END_TOKEN:
          break
  print(pred.shape)
  print(decoder_input_pred)
  
  return encoder_input, decoder_input_pred
  

In [ ]:
def predict(sentence):
  encode_input, decode_input = evaluate(sentence)
  
  t=model.predict([encode_input,decode_input])
  prediction=np.argmax(t,axis=2)
  print(prediction)
  print(prediction.shape)
  prediction=tf.squeeze(prediction,axis=0)  
  #prediction_sentence=tokenizer.decode(prediction)
  
  predicted_sentence=[]
  for i in prediction:
    if i>8510:
        break
    else:
        predicted_sentence.append(tokenizer.decode([i]))
    
        
  #predicted_sentence = tokenizer.decode(
    #[i for i in list(prediction) if i < 8511])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [ ]:
exclude=set(string.punctuation)
def preprocess_sentence(x):
    t=[]
    for i in range(len(x)):
        if x[i] not in exclude:
            t.append(x[i])
    #print(t)
    x3=''
    for i in range(len(t)):
        x3=x3+t[i]+''
    return x3

sentence='피곤하고 재미없다.'
sentence_1=preprocess_sentence(sentence)
print(sentence_1)

predict(sentence)

In [ ]:
predict('3박4일 놀러가고 싶다')#띄어쓰기에 영향받음
predict('카페 갈래')

In [ ]:
predict('12시 땡')
predict('PPL 심하네')

In [ ]:
predict('밥 먹어라')#끝음에 민감함

In [ ]:
sentence = preprocess_sentence('밥 먹었니?')
token_sentence=[]
sentence=tokenizer.encode(sentence)
token_sentence.append(sentence)
token_sentence=tf.keras.preprocessing.sequence.pad_sequences(token_sentence, maxlen=20, padding='post')
#token_sentence=tokenizer.encode(sentence)
print(type(token_sentence))
print(token_sentence)
#sentence = tf.expand_dims(token_sentence, axis=0)
print(sentence)

decoder_input_pred=np.zeros((1,max_length_a),dtype='int32')
print(decoder_input_pred)
decoder_input_pred[:,0]=START_TOKEN[0]
print(decoder_input_pred)


In [ ]:

token_sentence=tf.keras.preprocessing.sequence.pad_sequences([[486., 2091., 291.]], maxlen=20, padding='post')
print(token_sentence)
print(type(token_sentence))
print(token_sentence.shape)